## Using RNN frameworks for text sentiment classification

Here we aare going to use reccurent neural networks to classify the sentitment/emotion of the data GOemotions dataset which can be found here:
https://www.kaggle.com/datasets/shivamb/go-emotions-google-emotions-dataset

In [ ]:
# Imprting required libraries

import numpy as np
import pandas as pd

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence, text

from sklearn.model_selection import train_test_split

import nltk
nltk.download('wordnet')
import nltk
from nltk.stem import WordNetLemmatizer 

In [ ]:
data = pd.read_csv("../input/go-emotions-google-emotions-dataset/go_emotions_dataset.csv")

In [ ]:
data

In [ ]:
(data.columns)

In [ ]:
for i in range(3,len(data.columns)):
    print(data.columns[i])
    print(data.iloc[:,i].value_counts())

In [ ]:
data.drop("id", axis = 1, inplace = True)

In [ ]:
data.columns

In [ ]:
# combining 28 features into 6

l1 = [1, 2, 5, 6, 9, 14, 16, 18, 19, 21, 22, 24, 27] # Positive
l2 = [3, 4, 7, 10, 11, 12, 13, 15, 17, 20, 25, 26] # negative
l3 = [8, 23, 28] # neutral

In [ ]:
dct = {0 : 'Positive', 1 : 'Negative', 2 : 'Neutral'}

In [ ]:
dt = pd.DataFrame()

In [ ]:
# creating new data frame
dt['text'] = data['text'].values
dt['example_very_unclear'] = data['example_very_unclear']

In [ ]:
lt = np.zeros(shape=(data.shape[0], 3))
vals = data.iloc[:,2:].values

In [ ]:
len(vals[0])

In [ ]:
# combining features 
for index, col in enumerate(vals):
    flg = 0
    for i in l1:
        flg += col[i-1]
    if flg >= 1:
        lt[index][0] = 1
        continue
    for i in l2:
        flg += col[i-1]
    if flg >= 1:
        lt[index][1] = 1
        continue
    for i in l3:
        flg += col[i-1]
    if flg >= 1:
        lt[index][2] = 1
        continue

In [ ]:
lt[0]

In [ ]:
data.shape[0]

In [ ]:
nary = np.zeros(shape=(data.shape[0], 1))
len(nary[0])

In [ ]:
nary = np.zeros(shape=(data.shape[0]))

for i in range(3):
    for j in range(nary.shape[0]):
        nary[j] = int(lt[j][i])
#     print(nary)
    dt[dct[i]] = pd.DataFrame(nary)

In [ ]:
# fetching only raws that is clear
dt = dt[dt['example_very_unclear'] == False]

In [ ]:
# changing index
dt.index = range(dt.shape[0])

In [ ]:
data = dt

In [ ]:
data

In [ ]:
data = data.drop(data[data["example_very_unclear"] == True].index, axis = 0)

x = data.loc[:,"text"]
x = x.values
y = data.drop(["text"], axis = 1)
y = y.drop('example_very_unclear', axis = 1)
y = y.values

In [ ]:
# Lemmatization
lemmatizer = WordNetLemmatizer()

for i in range(len(x)):
    sentence = x[i]
    words = nltk.word_tokenize(sentence)
    lemmatized_sentence = ' '.join([lemmatizer.lemmatize(w) for w in words])
    x[i] = lemmatized_sentence
    

In [ ]:
# MAx words in a sentence

max_length = data['text'].apply(lambda x:len(str(x).split())).max()

In [ ]:
max_length

In [ ]:
# Tokenizing all words

tokenizer = text.Tokenizer()

tokenizer.fit_on_texts(list(x))

x_seq = tokenizer.texts_to_sequences(x)
x_pad = sequence.pad_sequences(x_seq, maxlen = max_length, pandding ="post")

word_ids = tokenizer.word_index

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_pad, y, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [ ]:
# Using GloVe(length = 200, due to memory restrictions) dataset for word embedding

glove = open("../input/glove6b200d/glove.6B.200d.txt")

# creating an embedding matrix for the embedding layer

embd_arrays = {}

for line in glove:
    line = line.split(" ")
    word = line[0]
    word_emd = np.asarray([integer for integer in line[1:]])
    embd_arrays[word] = word_emd

glove.close()

In [ ]:
# Preparing embedding matrix containing embedding vectors from glove

embed_matrix = np.zeros((len(word_ids) + 1, 200))
for word, i in list(word_ids.items()):
    embed_vector = embd_arrays.get(word)
    if embed_vector is not None:
        embed_matrix[i] = embed_vector

In [ ]:
# Creating the model 

model_lstm = Sequential()
model_lstm.add(Embedding(len(word_ids) + 1,
                 200,
                 weights=[embed_matrix],
                 input_length=max_length,
                 trainable=True))

model_lstm.add(Bidirectional(LSTM(100, dropout=0.3, recurrent_dropout=0.3)))
model_lstm.add(Dense(3, activation='sigmoid'))
model_lstm.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer='adam',metrics=['accuracy'])

model_lstm.summary()

In [ ]:
x_train

In [ ]:
model_L = keras.models.load_model("../input/lstm-models/lstm_v4.h5")

In [ ]:
x_train[0]

In [ ]:
y_train[0]

In [ ]:
x_train_2 = x_train[50000:60000]
y_train_2 = y_train[50000:60000]

In [ ]:
model_lstm.fit(x_train_2, y_train_2, epochs=10, batch_size=64, validation_split = 0.2)

In [ ]:
model_lstm.save('V3.h5')

In [ ]:
model_lstm.evaluate(x_test, y_test)

In [ ]:
model_L.fit(x_train, y_train, epochs=10, batch_size=64)

In [ ]:
model_L.predict([x_train[0]])

In [ ]:
y_train

In [ ]:
model_lstm.save('lstm_v6.h5')

In [ ]:
model_lstm.fit(x_train, y_train, epochs=10, batch_size=64)

In [ ]:
model_lstm.save('lstm_v7.h5')

In [ ]:
model_gru = Sequential()
model_gru.add(Embedding(len(word_ids) + 1,
                 200,
                 weights=[embed_matrix],
                 input_length=max_length,
                 trainable=True))

model_gru.add(GRU(100, dropout=0.3, recurrent_dropout=0.3))
model_gru.add(Dense(28, activation='sigmoid'))
model_gru.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer='adam',metrics=['accuracy'])

model_gru.summary()

In [ ]:
model.save('lstm_v1.h5')

In [ ]:
len(y_train), len(x_train)

In [ ]:
model_gru.fit(x_train, y_train, epochs=5, batch_size=64)